In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import *

In [3]:
from pyspark.ml.feature  import Tokenizer,RegexTokenizer,StopWordsRemover,CountVectorizer,OneHotEncoderEstimator,StringIndexer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
import pyspark.sql.functions as f

In [5]:
sc=pyspark.SparkContext()

In [6]:
spark= SparkSession \
       .builder \
       .appName("DICPA3") \
       .config("spark.some.config.option","some-value") \
       .getOrCreate() 

In [7]:
train_pd = pd.read_csv('train.csv')

In [8]:
mapping_pd = pd.read_csv('mapping.csv')


In [9]:
mapping=spark.createDataFrame(mapping_pd)


In [10]:
train=spark.createDataFrame(train_pd)

In [11]:
train.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [12]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")


train=regexTokenizer.transform(train)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

train=stopwordsRemover.transform(train)

model1 = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=15000)





In [13]:

train=model1.transform(train)

In [14]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel= idf.fit(train)


In [15]:
train = idfModel.transform(train)

In [16]:
mapping=spark.createDataFrame(mapping_pd,['id',"genre"])
mapgenre=mapping.select("genre","id").rdd.collectAsMap()

In [17]:
mapping.show()
count=mapping.count()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [18]:

train.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|(15000,[125,719,1...|(15000,[125,719,1...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|(15000,[15,47,53,...|(15000,[15,47,53,...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|(15000,[46,104,25...|(15000,[46,104,25...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|

In [19]:

def function1(g):
    x=[]
    genre=g[1:-1].split(",")  
    for i in genre:
        value=i.strip()[1:-1]
        x.append(mapgenre.get(value))
    x.sort()
    return x
myfunction1=f.udf(function1,ArrayType(IntegerType()))
train=train.withColumn("index",myfunction1("genre"))

train.select('index').show()

+----------------+
|           index|
+----------------+
|          [0, 5]|
|  [0, 4, 10, 17]|
|      [0, 4, 16]|
|             [1]|
|       [0, 5, 6]|
|   [0, 3, 4, 10]|
|       [0, 3, 7]|
|             [0]|
|[0, 1, 2, 8, 19]|
|    [12, 13, 15]|
|             [1]|
|    [0, 1, 5, 6]|
|             [1]|
|             [1]|
|             [7]|
|   [3, 6, 7, 18]|
|             [0]|
| [2, 3, 4, 6, 9]|
|          [0, 9]|
|             [0]|
+----------------+
only showing top 20 rows



In [20]:

def function2(g):
    y=1
    x=[]
    for i in range(0,count):
        x.append(0)
    
    
    for i in g:
        x[i]=1
        
    
    return x
myfunction2=f.udf(function2,ArrayType(IntegerType()))
traindata=train.withColumn("label",myfunction2("index"))
traindata.select('features','index','label').show()

+--------------------+----------------+--------------------+
|            features|           index|               label|
+--------------------+----------------+--------------------+
|(15000,[125,719,1...|          [0, 5]|[1, 0, 0, 0, 0, 1...|
|(15000,[15,47,53,...|  [0, 4, 10, 17]|[1, 0, 0, 0, 1, 0...|
|(15000,[46,104,25...|      [0, 4, 16]|[1, 0, 0, 0, 1, 0...|
|(15000,[46,76,158...|             [1]|[0, 1, 0, 0, 0, 0...|
|(15000,[15,53,80,...|       [0, 5, 6]|[1, 0, 0, 0, 0, 1...|
|(15000,[111,145,1...|   [0, 3, 4, 10]|[1, 0, 0, 1, 1, 0...|
|(15000,[15,114,14...|       [0, 3, 7]|[1, 0, 0, 1, 0, 0...|
|(15000,[17,24,76,...|             [0]|[1, 0, 0, 0, 0, 0...|
|(15000,[15,94,220...|[0, 1, 2, 8, 19]|[1, 1, 1, 0, 0, 0...|
|(15000,[1200,1649...|    [12, 13, 15]|[0, 0, 0, 0, 0, 0...|
|(15000,[76,116,15...|             [1]|[0, 1, 0, 0, 0, 0...|
|(15000,[76,690,16...|    [0, 1, 5, 6]|[1, 1, 0, 0, 0, 1...|
|(15000,[50,660,73...|             [1]|[0, 1, 0, 0, 0, 0...|
|(15000,[15,104,12...|  

In [21]:
test_pd = pd.read_csv('test.csv')
test=spark.createDataFrame(test_pd)
test.show()


+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [22]:
test=regexTokenizer.transform(test)

test=stopwordsRemover.transform(test)


test= model1.transform(test)

In [23]:
testdata = idfModel.transform(test)

In [24]:
testdata.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|(15000,[79,158,21...|(15000,[79,158,21...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|(15000,[213,1126,...|(15000,[213,1126,...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|(15000,[89,230,24...|(15000,[89,230,24...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|(15000,[183,534,6...|(15000,[183,534,6...|
|24165951|        Getting Even|A soldier-

In [25]:
x=[]

for i in range(0,count):
    
#     def parsePoint(line):
    
#         return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
#     parsedData = traindata.rdd.map(parsePoint)
#     part2model=LogisticRegressionWithLBFGS.train(parsedData)
    filename='./part2test'+str(i)
#    part2model.save(spark,filename)
    newmodel = LogisticRegressionModel.load(spark,filename)
    labelsAndPreds = testdata.rdd.map(lambda p: (p.movie_id, newmodel.predict(MLLibVectors.fromML(p.features))))
    x.append(labelsAndPreds.collect())

In [26]:
y={}
for i in x:
    for j in i:
        if j[0] in y:
            y[j[0]].append(j[1])
        else:
            y[j[0]]=[]
            y[j[0]].append(j[1])
for i in y:
    print(i,y[i])

1335380 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29062594 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9252321 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13455076 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
24165951 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
1925869 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10799612 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28238240 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
17124781 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28207941 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
19174305 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
18392317 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
34420857 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
4039635 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8034072 [1, 0, 0, 0, 0, 

5559264 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
9107230 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29680415 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2058218 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
6045984 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
11762841 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10483931 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
22822904 [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
31804235 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7740758 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
30138638 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3562033 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
14854661 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
10320498 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29082218 [1, 0, 1, 0, 0, 0

16166681 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
961542 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
14690165 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
4290149 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32021767 [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
23613543 [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
35124614 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25206110 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11671400 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
31460305 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
6414765 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
168491 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
22876558 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17942252 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3926453 [0, 1, 0, 0, 0, 0,

26613648 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30084641 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
21149183 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1348418 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
11089663 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
24142265 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
29462285 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1784731 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
20774714 [1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24882606 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
21035176 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8262764 [1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
61059 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4851941 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
29039930 [1, 0, 1, 0, 0, 0,

2626454 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
220521 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
22858004 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7452100 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
35013967 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
4369608 [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
34536204 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24501467 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8500485 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17957236 [1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
17050837 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
77856 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24828564 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
456587 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1665781 [1, 0, 1, 0, 0, 0, 0, 0

18488743 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6550500 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
1605358 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
26054640 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
15791592 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32631108 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13870077 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
34953366 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3864619 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
20181491 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
4021176 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11594636 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21868051 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
34383684 [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
15871827 [1, 0, 0, 1, 1,

14740084 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
20267384 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4324255 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
21867426 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
27457018 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
4294276 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
35493336 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
939519 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
3997782 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32748561 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2551631 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
8154771 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
14487270 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2170497 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
992419 [0, 1, 0, 0, 0, 0, 0,

7301880 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
22056357 [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
1668414 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
13577531 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1559551 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
20692991 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
11531376 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
669480 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2664825 [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1564174 [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3213020 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3557540 [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4327959 [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4199428 [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
34429476 [0, 1, 1, 0, 0, 0, 0, 

3940904 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2506530 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8974603 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
34394367 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
15974702 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1064199 [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10572944 [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
14537381 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8910297 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
9386023 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
36430069 [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
14066515 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2490751 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
35071703 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
7111027 [0, 0, 1, 0, 0, 0, 

16490737 [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1631497 [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1970341 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
525486 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10822975 [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
20479983 [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28344574 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10770671 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
25914398 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32401769 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7088713 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
3259315 [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
1840870 [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
34049319 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4600467 [0, 0, 0, 0, 0, 0, 

In [27]:
keypred={}

for i in y:
    x=""
    for j in y[i]:
        x=x+" "+str(j)
    x=x.strip()
    keypred[i]=x
for i in keypred:
    print(i,str(keypred[i]))

1335380 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
29062594 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
9252321 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
13455076 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
24165951 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
1925869 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10799612 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
28238240 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
17124781 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
28207941 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0
19174305 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0
18392317 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
34420857 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
4039635 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
8034072 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4016437 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0
1520023 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
24589422 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
35068740 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0
21132951 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
6575053 0 0 0 0 0 0 0 1 0 0

1548001 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
401839 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
35218175 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
33397799 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
5374788 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0
26131098 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11256773 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0
27305615 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
34160632 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
22773122 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2246413 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
16130074 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
11796155 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1
2396634 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
13949859 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
32142896 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
1192770 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3365240 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
35604925 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0
2547388 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
32860592 0 0 0 0 1 1 0 0 0 0 

19729385 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
2168542 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
10603679 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
23461433 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
885274 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1
2506380 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4179782 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
6584379 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
164401 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
64657 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1
6181075 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3145613 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
7451262 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0
30865811 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3227389 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0
2506836 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
21690482 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
691156 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1
2006189 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
25106279 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
801722 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

21607739 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2015137 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0
15469761 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0
21436715 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
21744986 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
33575471 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
15749011 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0
10906825 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
15499614 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
3086340 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
8359241 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
32862586 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
18850952 1 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
33322482 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
21076018 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0
27947539 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
23670000 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2345042 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
3230593 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
9706751 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
32833838 1 0 0 1 0 1 0 1 0

29244961 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0
4894893 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0
31992423 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
149789 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
8677111 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
5884145 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0
19863623 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
1954311 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
31325794 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
7056998 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
17401665 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11541829 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
17595675 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
29495341 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
14143380 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0
10873860 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
3030423 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1
12813525 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
30099851 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
2511741 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0
804933 0 0 0 1 0 1 0 1 0 0 0 

5826081 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2107054 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
10286685 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
11562799 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0
21225053 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
3956816 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
33245032 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
594257 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
14144320 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
10605328 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1
14757585 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
12224218 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
67371 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
31230287 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
24041290 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0
1205454 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
33447795 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
2249957 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
7145079 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
6838440 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
28870239 0 0 0 0 0 0 0 1 1 0 0 0

6550694 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
10036523 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0
3104623 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
18895115 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
23600571 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
2721086 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
6795118 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
24110107 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
17792701 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
13514560 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0
28645571 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0
19091638 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10131472 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
21956466 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
28807164 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
10625487 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0
36057874 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1166394 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
3735041 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
34430482 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
605198 1 0 0 0 0 0 0 0 0 0

10386609 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10473518 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
14336880 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
26084706 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
28995043 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3705395 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0
17041756 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
12532758 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
14964780 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0
1210091 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0
5995554 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
5737121 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
31499930 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
14503994 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
32669119 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
33709032 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
6302125 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
6635409 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0
27504477 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
8097289 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
2583808 1 0 0 1 1 0 0 0 1 0

25747315 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1273572 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
2510700 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0
13787959 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
20993667 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
9271903 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2636349 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
25283935 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
29982473 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
18589876 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12159008 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
232711 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
20122638 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
2216810 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
14525860 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0
32428439 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
9556173 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
14818617 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11122446 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
31439778 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0
10649757 1 1 1 0 0 0 0 0 1 0

In [28]:
import csv

with open ('output2.csv','w',newline="") as mfile:
    writer=csv.writer(mfile)
    writer.writerow(["movie_id","predictions"])
    for key , value in keypred.items():
        writer.writerow([key,value])